In [12]:
import xarray as xr
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from os.path import join

from holodecml.data import load_unet_datasets_xy_1to25, load_train_patches_1to25
from holodecml.losses import unet_loss_xy


In [2]:
path_data = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/"
num_particles = [1,2,3,4,5,6,7,8,9,10,"12-25"]
output_cols = ["x", "y", "z", "d", "hid"]
subset = [500,500,500,500,500,500,500,500,500,500,6500]
scaler_out = MinMaxScaler()
bin_factor = 10
rad = 64
input_col = "image"


In [3]:
train_inputs,\
train_outputs,\
valid_inputs,\
valid_outputs = load_unet_datasets_xy_1to25(path_data,
                                            num_particles,
                                            output_cols,
                                            subset,
                                            bin_factor)


In [4]:
model_path = "/glade/p/cisl/aiml/ggantos/holodec/unet/jnet_10_1to25/"

In [5]:
model = tf.keras.models.load_model(model_path, custom_objects={'unet_loss_xy': unet_loss_xy})
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 400, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 600, 400, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 600, 400, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 600, 400, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
train_outputs_pred = model.predict(np.expand_dims(train_inputs, axis=-1),
                                   batch_size=128)


In [10]:
train_outputs_pred_da = xr.DataArray(train_outputs_pred,
                                     coords={"hid": np.arange(train_outputs_pred.shape[0]),
                                             "x": np.arange(train_outputs_pred.shape[1]),
                                             "y": np.arange(train_outputs_pred.shape[2]),
                                             "output": ["p"]},
                                     dims=("hid", "x", "y", "output"),
                                     name="train_pred_scaled")
train_outputs_pred_da.to_netcdf(join(model_path, "train_outputs_pred.nc"))


In [15]:
valid_outputs_pred = model.predict(np.expand_dims(valid_inputs, axis=-1),
                                   batch_size=128)


In [16]:
valid_outputs_pred_da = xr.DataArray(valid_outputs_pred,
                                     coords={"hid": np.arange(valid_outputs_pred.shape[0]),
                                             "x": np.arange(valid_outputs_pred.shape[1]),
                                             "y": np.arange(valid_outputs_pred.shape[2]),
                                             "output": ["p"]},
                                     dims=("hid", "x", "y", "output"),
                                     name="valid_pred_scaled")
valid_outputs_pred_da.to_netcdf(join(model_path, "valid_outputs_pred.nc"))


In [17]:
train_outputs_pred.shape

(11500, 60, 40, 1)